<a href="https://colab.research.google.com/github/giacomogreggio/Algoritmi_avanzati/blob/master/Alg_Av_HW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Downloading Dataset

In [34]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
dataset_path="/content/drive/My Drive/Colab Notebooks/algoritmi_avanzati/Homework_1/dataset/input_random_01_10.txt"

# Defining the data structures

In [0]:
class Edge:
  def __init__(self, node1,node2, weight=None):
    self.node1=node1
    self.node2=node2
    self.weight=weight

In [0]:
def import_graph(dataset_path):
  dataset_file=open(dataset_path,"r")
  content=dataset_file.read()
  dataset=content.split()

  n_nodes=int(dataset.pop(0))
  n_edges=int(dataset.pop(0))
  edge_list=[]
  adiacence_list={}
  for i in range(0,n_nodes+1):
    adiacence_list[i]=[]
  
  i=0
  while i<len(dataset):
    new_edge=Edge(int(dataset[i]),int(dataset[i+1]),int(dataset[i+2]))
    edge_list.append(new_edge)
    adiacence_list[int(dataset[i])].append(new_edge)
    i=i+3
  return edge_list, adiacence_list, n_nodes, n_edges

In [0]:
edge_list, adiacence_list, n_nodes, n_edges = import_graph(dataset_path)